In [1]:
from pathlib import Path
import sys
import re
import pandas as pd
import shutil
import os
from multiprocessing import Pool
import prody
import pickle
ABPRED_DIR = Path.cwd().parent
if ABPRED_DIR not in sys.path:
    sys.path.append(str(ABPRED_DIR))
from AbPred.FoldX import FoldX
from modeller import *
from modeller.optimizers import molecular_dynamics, conjugate_gradients
from modeller.automodel import autosched


In [2]:
# Skempi dataframe of selected AB complexes
ab_data_singleMut = pd.read_csv('../data/skempi_ABlike_singleMut.Final.csv',index_col=0)
ab_data_singleMut.sort_values('#Pdb',inplace=True)

# Paths data
PDBS_DIR = Path("skempiAB-modeller/")
skempiAB_pdbs = list(PDBS_DIR.glob("*.pdb"))

In [3]:
ab_data_singleMut

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,koff_wt_parsed,dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version,ddG,pdb_mutation
282,1AHW_AB_C,YC156A,YC145A,COR,AB/AG,AB/AG,2.100000e-06,2.1E-06,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,2,1.647477,1AHW_AB_C_YC145A
292,1AHW_AB_C,VC198A,VC187A,RIM,AB/AG,AB/AG,2.000000e-09,"2,00E-09",3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,-0.136043,1AHW_AB_C_VC187A
291,1AHW_AB_C,TC197A,TC186A,RIM,AB/AG,AB/AG,3.300000e-08,3.3E-08,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,0.582683,1AHW_AB_C_TC186A
289,1AHW_AB_C,DC178A,DC167A,RIM,AB/AG,AB/AG,1.500000e-09,1.5E-09,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,-0.209798,1AHW_AB_C_DC167A
293,1AHW_AB_C,NC199A,NC188A,RIM,AB/AG,AB/AG,2.100000e-08,2.1E-08,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,0.466803,1AHW_AB_C_NC188A
287,1AHW_AB_C,TC170A,TC159A,SUP,AB/AG,AB/AG,2.200000e-08,2.2E-08,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,0.478730,1AHW_AB_C_TC159A
285,1AHW_AB_C,TC167A,TC156A,COR,AB/AG,AB/AG,3.000000e-09,"3,00E-09",3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,-0.032089,1AHW_AB_C_TC156A
283,1AHW_AB_C,YC157A,YC146A,SUP,AB/AG,AB/AG,1.400000e-10,1.4E-10,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,-0.817824,1AHW_AB_C_YC146A
288,1AHW_AB_C,LC176A,LC165A,RIM,AB/AG,AB/AG,1.800000e-08,1.8E-08,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1,0.427282,1AHW_AB_C_LC165A
4865,1BJ1_HL_VW,YH54A,YH54A,RIM,AB/AG,AB/AG,3.200000e-08,3.2E-08,3.400000e-09,3.4E-09,...,NaN,NaN,NaN,NaN,NaN,NaN,SPR,2,0.574794,1BJ1_HL_VW_YH54A


# Parse only protein pdb data

In [ ]:
for pdb in skempiAB_pdbs:
    
    parser = prody.parsePDB(pdb)
    protein = parser.select("protein")
    prody.writePDB(str(pdb)[:-4]+".protein",protein)

In [ ]:
skempiAB_protein= list(PDBS_DIR.glob("*protein.pdb"))

#  Run mutation protocol by modeller

In [ ]:
amino3to1dict = {'ALA': 'A','CYS': 'C','ASP': 'D','GLU': 'E','PHE': 'F','GLY': 'G',
                 'HIS': 'H','ILE': 'I','LYS': 'K','LEU': 'L','MET': 'M','ASN': 'N',
                 'PRO': 'P','GLN': 'Q','ARG': 'R','SER': 'S','THR': 'T','VAL': 'V',
                 'TRP': 'W','TYR': 'Y'}

def convertAAcode(m_splits):
    """Function to get the 3letters aa in amino3to1dict
    using the split_mutations() output"""
    for k,v in amino3to1dict.items():
        if v == m_splits[0]:
            return k
        
def split_mutations(mut_str):
    """Function to split a [mutation] string, searching the mutaion patter and return groups.
    Output is a list with 3 elements [wt_aa,aa_number,mut_aa] """
    
    search_mut = re.search("([A-Z])([A-Z])([0-9]+[A-Z]*)([A-Z])",mut_str,flags=re.I)
    m_splits = search_mut.groups()
    
    return list(m_splits)

In [ ]:

def mutate_model(modelname,respos,restyp,chain,random_seed):
    
    #
    #  mutate_model.py
    #
    #     Usage:   python mutate_model.py modelname respos resname chain > logfile
    #
    #     Example: python mutate_model.py 1t29 1699 LEU A > 1t29.log
    #
    #
    #  Creates a single in silico point mutation to sidechain type and at residue position
    #  input by the user, in the structure whose file is modelname.pdb
    #  The conformation of the mutant sidechain is optimized by conjugate gradient and
    #  refined using some MD.
    #
    #  Note: if the model has no chain identifier, specify "" for the chain argument.
    #


    def optimize(atmsel, sched):
        #conjugate gradient
        for step in sched:
            step.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)
        #md
        refine(atmsel)
        cg = conjugate_gradients()
        cg.optimize(atmsel, max_iterations=200, min_atom_shift=0.001)


    #molecular dynamics
    def refine(atmsel):
        # at T=1000, max_atom_shift for 4fs is cca 0.15 A.
        md = molecular_dynamics(cap_atom_shift=0.39, md_time_step=10.0,
                                md_return='FINAL')
        init_vel = True
        for (its, equil, temps) in ((200, 20, (150.0, 250.0, 400.0, 700.0, 1000.0,1500.0)),
                                    (2000, 200,
                                     (1500.0,1000.0, 800.0, 600.0, 500.0, 400.0, 300.0))):
            for temp in temps:
                md.optimize(atmsel, init_velocities=init_vel, temperature=temp,
                             max_iterations=its, equilibrate=equil)
                init_vel = False


    #use homologs and dihedral library for dihedral angle restraints
    def make_restraints(mdl1, aln):
        rsr = mdl1.restraints
        rsr.clear()
        s = selection(mdl1)
        for typ in ('stereo', 'phi-psi_binormal'):
            rsr.make(s, restraint_type=typ, aln=aln, spline_on_site=True)
        for typ in ('omega', 'chi1', 'chi2', 'chi3', 'chi4'):
            rsr.make(s, restraint_type=typ+'_dihedral', spline_range=4.0,
                    spline_dx=0.3, spline_min_points = 5, aln=aln,
                    spline_on_site=True)

    #first argument
    #modelname, respos, restyp, chain, = sys.argv[1:]


    log.verbose()

    # Set a different value for rand_seed to get a different final model
    env = environ(rand_seed=random_seed)

    env.io.hetatm = False
    #soft sphere potential
    env.edat.dynamic_sphere=False
    #lennard-jones potential (more accurate)
    env.edat.dynamic_lennard=True
    env.edat.contact_shell = 4.0
    env.edat.update_dynamic = 0.39

    # Read customized topology file with phosphoserines (or standard one)
    env.libs.topology.read(file='$(LIB)/top_heav.lib')

    # Read customized CHARMM parameter library with phosphoserines (or standard one)
    env.libs.parameters.read(file='$(LIB)/par.lib')


    # Read the original PDB file and copy its sequence to the alignment array:
    mdl1 = model(env, file=modelname)
    ali = alignment(env)
    ali.append_model(mdl1, atom_files=modelname, align_codes=modelname)

    #set up the mutate residue selection segment
    s = selection(mdl1.chains[chain].residues[respos])

    #perform the mutate residue operation
    s.mutate(residue_type=restyp)
    #get two copies of the sequence.  A modeller trick to get things set up
    ali.append_model(mdl1, align_codes=modelname)

    # Generate molecular topology for mutant
    mdl1.clear_topology()
    mdl1.generate_topology(ali[-1])


    # Transfer all the coordinates you can from the template native structure
    # to the mutant (this works even if the order of atoms in the native PDB
    # file is not standard):
    #here we are generating the model by reading the template coordinates
    mdl1.transfer_xyz(ali)

    # Build the remaining unknown coordinates
    mdl1.build(initialize_xyz=False, build_method='INTERNAL_COORDINATES')

    #yes model2 is the same file as model1.  It's a modeller trick.
    mdl2 = model(env, file=modelname)

    #required to do a transfer_res_numb
    #ali.append_model(mdl2, atom_files=modelname, align_codes=modelname)
    #transfers from "model 2" to "model 1"
    mdl1.res_num_from(mdl2,ali)

    #It is usually necessary to write the mutated sequence out and read it in
    #before proceeding, because not all sequence related information about MODEL
    #is changed by this command (e.g., internal coordinates, charges, and atom
    #types and radii are not updated).

    mdl1.write(file=modelname+restyp+respos+'.tmp')
    mdl1.read(file=modelname+restyp+respos+'.tmp')

    #set up restraints before computing energy
    #we do this a second time because the model has been written out and read in,
    #clearing the previously set restraints
    make_restraints(mdl1, ali)

    #a non-bonded pair has to have at least as many selected atoms
    mdl1.env.edat.nonbonded_sel_atoms=1

    sched = autosched.loop.make_for_model(mdl1)

    #only optimize the selected residue (in first pass, just atoms in selected
    #residue, in second pass, include nonbonded neighboring atoms)
    #set up the mutate residue selection segment
    s = selection(mdl1.chains[chain].residues[respos])

    mdl1.restraints.unpick_all()
    mdl1.restraints.pick(s)

    s.energy()

    s.randomize_xyz(deviation=4.0)

    mdl1.env.edat.nonbonded_sel_atoms=2
    optimize(s, sched)

    #feels environment (energy computed on pairs that have at least one member
    #in the selected)
    mdl1.env.edat.nonbonded_sel_atoms=1
    optimize(s, sched)

    s.energy()

    dope_energy = mdl1.assess_normalized_dope()
    #give a proper name
    mdl1.write(file=modelname[:-4]+"."+chain+respos+amino3to1dict[restyp]+'.wt.pdb')

    #delete the temporary file
    os.remove(modelname+restyp+respos+'.tmp')
    
    return dope_energy


In [ ]:
def mainModmut(row):
    pdb_partners = row['#Pdb']
    pdbid = pdb_partners.split('_',maxsplit=1)[0]
    partners = pdb_partners.split('_',maxsplit=1)[1]    
    mutations_cleaned = row['Mutation(s)_cleaned']
    pdbname = '{}.protein.pdb'.format(pdbid)
    
    # get argument for mutate function
    modelfile = PDBS_DIR/Path(pdbname)
    mutation_split = split_mutations(mutations_cleaned)
    restyp = convertAAcode(mutation_split[-1])
    chain = mutation_split[1]
    respos = mutation_split[2]
    
    dope_energy =  mutate_model(modelname=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    return dope_energy

def mainModwt(row):
    pdb_partners = row['#Pdb']
    pdbid = pdb_partners.split('_',maxsplit=1)[0]
    partners = pdb_partners.split('_',maxsplit=1)[1]    
    mutations_cleaned = row['Mutation(s)_cleaned']
    pdbname = '{}.protein.{}.mut.pdb'.format(pdbid,mutations_cleaned[1:])
    
    # get argument for mutate function
    modelfile = PDBS_DIR/Path(pdbname)
    mutation_split = split_mutations(mutations_cleaned)
    restyp = convertAAcode(mutation_split[0])
    chain = mutation_split[1]
    respos = mutation_split[2]
    
    dope_energy =  mutate_model(modelname=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    return dope_energy

In [ ]:
for i,row in ab_data_singleMut.iterrows():
    pdb_partners = row['#Pdb']
    pdbid = pdb_partners.split('_',maxsplit=1)[0]
    partners = pdb_partners.split('_',maxsplit=1)[1]    
    mutations_cleaned = row['Mutation(s)_cleaned']
    pdbname = '{}.protein.{}.mut.pdb'.format(pdbid,mutations_cleaned[1:])

    # get argument for mutate function
    modelfile = PDBS_DIR/Path(pdbname)
    mutation_split = split_mutations(mutations_cleaned)
    # wildtype res
    restyp = convertAAcode(mutation_split[0])
    chain = mutation_split[1]
    respos = mutation_split[2]

    dope_energy =  mutate_model(modelname=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    break

In [ ]:
CWD = Path.cwd()

energy_mut = list()
try:
    #os.chdir('skempiAB-modeller')
    with Pool(12) as p:
        dope_energy = p.map(mainModmut, [row for i,row in ab_data_singleMut.iterrows()])
        energy_mut.append(dope_energy)
finally:
    os.chdir(CWD)

In [ ]:
with open('energy_dope_modellerSkempiABmut.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(energy_mut, f, pickle.HIGHEST_PROTOCOL)

In [3]:
skempiAB_mutant= list(PDBS_DIR.glob("*mut.pdb"))

## Model from mutant to wildtype

In [ ]:
CWD = Path.cwd()

energy_wt = list()
try:
    #os.chdir('skempiAB-modeller')
    with Pool(12) as p:
        dope_energy = p.map(mainModwt, [row for i,row in ab_data_singleMut.iterrows()])
        energy_wt.append(dope_energy)
finally:
    os.chdir(CWD)

In [ ]:
with open('energy_dope_modellerSkempiABwt.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(energy_wt, f, pickle.HIGHEST_PROTOCOL)

In [6]:
skempiAB_wt= list(PDBS_DIR.glob("*wt.pdb"))

## Extract only ATOM data, rewrite models
(problme originated becouse first run with modeller set env.io.hetatm True )

In [18]:
from biopandas.pdb import PandasPdb

for mutant in skempiAB_mutant:
    ppdb = PandasPdb().read_pdb(str(mutant))
    
    ppdb.to_pdb(str(mutant),records=['ATOM','OTHERS'])
    

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/biopandas/pdb/pandas_pdb.py:519: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(dfs)


In [19]:
for mutant in skempiAB_wt:
    ppdb = PandasPdb().read_pdb(str(mutant))
    
    ppdb.to_pdb(str(mutant),records=['ATOM','OTHERS'])
    

/Users/vfica/miniconda3/envs/bio/lib/python3.6/site-packages/biopandas/pdb/pandas_pdb.py:519: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(dfs)


# Repair PDb mut

#### This was done on clsuter,  4 Repair run for each structure